<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Building Agentic AI Applications with LLMs</h1>
<h2><b>Tangent 1:</b> Trying Out CrewAI</h2>
<br>


**This is the first of several tangents** in which we will talk about something that is important to know about, but will not be focused on much in this course. Consider why we're not focusing on this material, but try to appreciate it as an on-ramp to something you're likely to run into going forward.

In notebook 1a, we actually defined our own tiny agent system, if only to properly allow two or more non-human agents a chance to talk to each other in a reasonable fashion. In this notebook, we will briefly look at an agent framework that is especially popular and useful for modeling persona-based agents, and is quite easy to set up for this problem: [**CrewAI**](https://www.crewai.com/open-source)!

### **Learning Objectives:**

**In this notebook, we will:**

- Learn a bit about pre-built agent frameworks that implements our earlier abstractions with a spin and more coverage.
- Investigate CrewAI specifically and consider how we can replicate our previous teacher-student dialog.

<hr><br>

### Defining An Agent Framework

Whether you fully appreciate it or not, both of the systems demonstrated in notebook 1a were, in fact, **agent systems**. They were software systems where at least one software component semantically percieved the environment and responded to the best of their ability to satisfy a vague objective.
- The basic chat loop was literally just a loop that sampled responses from the LLM and the user in a loop. The environment is the message bus, the agents are you and the LLM, and the process was represented by the responses flowing between the agents.
- The local-perspective systems, also defined in a loop, were very similar but had better support for modeling multiple destinct personas. They mapped from some global state system into something the LLM was suited to handle, and mapped back up to the global state in the same manner.

We were technically relying on the LangChain software stack to connect to our model below a plethora of abstractions, but we really were just using some simple primitives to make our agent systems work. The real interesting parts involved the organization of these primitives to create synergized components that made it easy for our agents - or even humans - to communicate appropriately. 

**In this notebook, we will briefly introduce CrewAI as a potential framework of interest.** Though the course will not use CrewAI much - and we will explain why shortly - it is important to understand:
- What CrewAI is.
- What problems CrewAI solves.
- Why might people choose to use it.

<br>

## **What is CrewAI?**

The following is a direct exerpt from the [**Official CrewAI Documentation**](https://docs.crewai.com/introduction) (sampled 2/21/2025):

<img src="images/crewai-purpose.png" style="width: 800px"/>

<!-- > **CrewAI is a cutting-edge framework for orchestrating autonomous AI agents.**
> 
> CrewAI enables you to create AI teams where each agent has specific roles, tools, and goals, working together to accomplish complex tasks.
>
> Think of it as assembling your dream team - each member (agent) brings unique skills and expertise, collaborating seamlessly to achieve your objectives. -->

As advertised, **CrewAI** is a well-built and general-purpose multi-agent framework, which roughly means it has:
- A mechanism for communication that it enforces.
- Some core workflows that it advertises to streamline.
- Some primitives defined to make those workflows easy to execute on.
- Paths towards productionalization for multi-tenant and concurrent execution. (*more on that later*)

In the section below, we will take advantage of some of its built-in primitives and investigate how this system operates and consider when we might use it.

#### **The CrewAI Mind Map**

Like all frameworks, CrewAI enforces some opinions about how an agent system should be structured/which kinds it best supports as first-class abstractions. Here is the latest version of their working mind-map for those taking a first-look at this framework.

<img src="images/crewai-mindmap.png" style="width: 800px"/>

Whenever you see something like this, understand that this is **a potential way** to think about the agent abstractions. Every potential option has its pros and cons, and there is a reason why we think it's important to advocate for CrewAI while also not teach explicitly to a specific option.

To be clear, their abstractions are more than enough to execute on the processes covered in the scope of this course, and we encourage you to try it out for yourself with more involved use-cases after the class!

#### **Defining Our LLM Client**

Though CrewAI and LangChain do have some shared integrations and compatability layers, CrewAI out-of-the-box likes to follow its own definition of the LLM client which is different from LangChain's. 

At the end of the day, they both do roughly the same things but implement different interfaces for the two frameworks to use. Obligatorily, we will then need to construct our interface a bit differently:

In [ ]:
from crewai import LLM

llm = LLM(
    model="nvidia_nim/meta/llama-3.1-8b-instruct",   ## Provider Class / Model Published / Model Name
    base_url="http://nim-llm:8000/v1",               ## Url to send your request to (ChatNVIDIA accepts env variable)
    temperature=0.7,
    api_key="PLACEHOLDER",                           ## API key is required by default.
)

llm.call(messages=[{"role": "user", "content": "What is the capital of France?"}])  ## Call, not "invoke"

#### **Defining Our "Chain Primitives"**

In LangChain, the runnables interface allows us to trivially chain multiple components together to chain buffers and/or simple invocations. Recall the ubiquitous `prompt | llm | StrOutputParser()` chain, and note that we will explore more interesting byproducts of these abstractions later. 

In CrewAI, many of their core primitives are more purpose-built to represent very specific mechanisms which interact with the agentic communication buffer in very well-defined ways. For example, the following cells show a typical construction of a minimal CrewAI `Crew`, or agent pool that work towards an objective:

> You can define one or more CrewAI [**`Agents`**](https://docs.crewai.com/concepts/agents), which are persona-based agents that communicates with other Agents. Combined with the `Prompts` utility abstraction, the `task_execution()` method gives you a base prompt for the agent (which can be added to by other mechanisms later).
> 
> In contrast, the [**`Task`**](https://docs.crewai.com/concepts/tasks) abstraction specifies actual directives for your agents to execute on. This requires a different set of arguments, encapsulates `Agent` entities to communicate which ones can work on the process, and computes an appropriate prompt component via the `.prompt()` method.
> 
> And to top it off, the [**`Crew`**](https://docs.crewai.com/concepts/crews) abstraction contains both `Task`s and `Agent`s, and allows them to communicate (via a `Process` class) in a sequential or hierarchical manner to achieve the list of `Tasks`.

**Said more plainly, the CrewAI abstraction has you:**
- **Defining agents** that have personas, backgrounds, and generic goals.
- **Defining tasks** that can be executed by a subset of agents in some manner.
- **Defining crews** of agents that work on groups of tasks with various witness mechanisms.

And this leads to control flow decisions and prompt injections that end up reaching your LLM endpoint, and the resulting responses help to guide both the conversational and executional environments.

### Seeing Some Code

We've discussed the typical CrewAI workflow, so let's see how that maps to actual code:

In [ ]:
from crewai import Agent
from crewai.utilities import Prompts

## - You can define one or more CrewAI `Agent`s, which are persona-based agents that communicates with other Agents.
##     - Combined with the `Prompts` utility abstraction, the `task_execution()` method gives you a base prompt for the agent 
##       (which can be added to by other mechanisms later).

## https://docs.crewai.com/concepts/agents#direct-code-definition

teacher_agent = Agent(
    role='Teacher',
    goal="Help students with concerns and make sure they are learning their material well.",
    backstory=(
        "You are a computer science teacher in high school holding office hours, and you have a meeting."
        " This is the middle of the semester, and various students have various discussion topics across your classes."
        " You are having a meeting right now. Please engage with the student."
    ),
    verbose=True,     ## Enable detailed execution logs for debugging
    memory=True,
    llm=llm,
)

student_agent = Agent(
    role='Student',
    goal="Be a good student while also maintaining personal interests and a healthy social life.",
    backstory=(
        "You are taking Dr. John's intro to algorithms course and are struggling with some of the homework problems."
    ),
    verbose=True,     ## Enable detailed execution logs for debugging
    memory=True,
    llm=llm,
)

print(Prompts(agent=teacher_agent).task_execution())
print("*" * 64)
print(Prompts(agent=student_agent).task_execution()["prompt"])

This example uses the default prompt template with language that can be found [here](https://github.com/crewAIInc/crewAI/blob/main/src/crewai/translations/en.json). The prompt can be customized to achieve specific behaviors. 

In [ ]:
from crewai import Task

## - In contrast, the `Task` abstraction specifies actual directives for your agents to execute on.
##     - This requires a different set of arguments, encapsulates `Agent` entities to communicate which ones can work
##       on the process, and computes an appropriate prompt component via the `.prompt()` method.

## https://docs.crewai.com/concepts/tasks#direct-code-definition-alternative

teacher_task = Task(
    description="Engage in dialog to help the student out.",
    expected_output="Conversational output that is supportive and helpful.",
    agent=teacher_agent,
    async_execution=False,
    # human_input=True,     ## Human-in-the-loop mechanism to correct the agent responses 
)

student_task = Task(
    description="Meet with your teacher to help you understand class material.",
    expected_output="Conversational responses",
    agent=student_agent,
    async_execution=False,
    # human_input=True,     ## Human-in-the-loop mechanism to correct the agent responses 
)

teacher_task.prompt()

`Agent`s and `Task`s can also be initialized with `tools`, which we will discuss more later. 

In [ ]:
from crewai import Crew, Process

## - And to top it off, the `Crew` abstraction contains both `Task`s and `Agent`s, and allows them to communicate 
##   (via a `Process` class) in a sequential or hierarchical manner to achieve the list of `Tasks`.

chatbot_crew = Crew(
    ## Shift state between teacher and student 4 times (i.e. t->s->t->s->...->s)
    agents=[teacher_agent, student_agent] * 4,
    tasks=[teacher_task, student_task] * 4,
    process=Process.sequential,     ## By default, tasks in CrewAI are managed through a sequential process. However,
                                    ##  adopting a hierarchical approach allows for a clear hierarchy in task management,
                                    ##  where a ‘manager’ agent coordinates the workflow, delegates tasks, and validates
                                    ##  outcomes for streamlined and effective execution. Configuring the manager_llm
                                    ##  parameter is crucial for the hierarchical process. 
    verbose=True,
)

In [ ]:
## Kick off the routine. If there are any {var}s in an agent/task prompt, you can specify inputs={'var': value, ...}
chatbot_crew.kickoff()

<hr><br>

### **Reflextion:** Is This Better Than LangChain?

***Sometimes yes, sometimes no!***
- For general LLM engineering, **the primitives offered by LangChain are much more flexible.** There are many modules and compatability layers outside of what this course will cover, and they can be used to make near-arbitrary data pipelines with great hidden properties that help with end-game productization.
- For agent applications that leverage persona-based systems, **CrewAI is likely the easiest entrypoint when you want to deploy groups of easy-to-specify agents.** You can already see that there is a lot of baked-in assumptions, and checking the parameter list will reveal various customization options which help to alleviate the pain of system specification and a decent bit of boilerplate.
- For more custom applications that require more involved state management systems, **LangGraph is another great option which we will take advantage of later.** This framework makes it easy to go deep into customization land while still sticking to the core abstraction, but it also generally requires a better understanding of agent system design and therefore has a higher learning curve. 

From an entry-point into agents perspective, it could be argued that the CrewAI framework is easier to get started with, since it pidgeonholes you into some specific workflow paradigms. All of these paradigms COULD be made with the primitive components offered by LangChain OR LangGraph, but there's significant value in an opinionated framework that makes the hard decisions for you and just lets you tap into the abstraction without too much trouble. For this reason, we will try to point out to **CrewAI** example solutions when we see it helping with the course narrative.

<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>